# My fine tuning
* Use n2c2 data

In [ ]:
# !pip install datasets transformers[sentencepiece]

In [ ]:
#import data science packages
import pandas as pd
import numpy as np
# import seaborn as sns
#dl imports
from datasets import load_dataset, Dataset, ClassLabel, load_from_disk, DatasetDict, load_metric
from huggingface_hub import notebook_login
from transformers import pipeline

import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
# read the full data
full_df = pd.read_csv('t20180910_wb20wa40.csv')
full_df.head()
print(full_df.shape)

In [ ]:
alldata_ds = Dataset.from_pandas(full_df)
alldata_ds = alldata_ds.class_encode_column('label' )
alldata_ds = alldata_ds.map(lambda x: {'ae_label2': alldata_ds.features['label'].int2str(x['label'])})
# alldata_ds.features['label'].int2str(0)
data_ds = alldata_ds.train_test_split(test_size=0.4, seed=1)
print(data_ds["test"]['label'])

In [ ]:
# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
#define tokenizing function
def tokenize_inputs(example):
    return tokenizer(example['excerpt'], truncation=True)
    
#do the tokenizing using map function
tokenized_ds = data_ds.map(tokenize_inputs, batched=True,
                           remove_columns = ['filename', 'posDrug', 'posAE'])
print(len(tokenized_ds['train']['input_ids']))
len(tokenized_ds['train']['input_ids'][0])
input0_tokens = tokenizer.convert_ids_to_tokens(tokenized_ds['train']['input_ids'][0])
no_classes= data_ds['train'].features['label'].num_classes
no_classes

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

id2label = {ind:label for ind, label in enumerate(data_ds['train'].features['label'].names)}
label2id = {label:ind for ind, label in id2label.items()}
print(id2label)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',
                                                           num_labels = no_classes,
                                                           id2label=id2label,
                                                           label2id=label2id)
# model.name_or_path

In [ ]:
training_arg = TrainingArguments("test-trainer",
                                 logging_strategy='epoch',
                                 evaluation_strategy = 'epoch',
                                 save_strategy = 'epoch', 
                                 load_best_model_at_end = True,
                                 metric_for_best_model='fscore',
                                 greater_is_better=True,
                                 report_to = 'all',
                                 per_device_train_batch_size = 30,
                                 per_device_eval_batch_size = 30, 
                                 num_train_epochs = 5,
                                 seed = 100
                                 )
#                                push_to_hub=True,
#                                hub_model_id = "choil/testFineTune"

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    #get predictions by using index of max logit
    predictions = np.argmax(logits, axis=-1)
    
    #calculate classification report
    perfs = precision_recall_fscore_support(labels, predictions, average='macro', zero_division=0)
    perf_dict = dict(zip(['precision', 'recall', 'fscore'], perfs[:3]))
    
    #return dictionary
    return perf_dict

In [ ]:
trainer = Trainer(model = model,
                  args=training_arg,
                  data_collator = data_collator,
                  tokenizer=tokenizer,
                  train_dataset = tokenized_ds['train'],
                  eval_dataset = tokenized_ds['test'],
                  compute_metrics = compute_metrics
                  )
trainer.train()

In [ ]:
# eval_ds = trainer.evaluate(tokenized_ds['test'])

In [ ]:
preds = trainer.predict(tokenized_ds['test'])
pred_df = pd.DataFrame(preds.predictions)
print(data_ds["test"])
test_df = pd.DataFrame(data_ds["test"])
# subdata_df = full_df.drop(['excerpt'], axis=1)
predtest_df = pd.concat([test_df, pred_df], axis=1)
# predtest_df.head()

In [ ]:
predtest_df.to_csv('t20180910_wb20wa40_b30_pred.csv')